In [10]:
import pyreadr
transactions_df = pyreadr.read_r("/home/spadef/data/IDEA_DeFi_Research/Data/Lending_Protocols/Aave/V3/Polygon/transactionsFinSurvival.rds")[None]
# transactions_df.to_csv("/home/spadef/data/IDEA_DeFi_Research/Data/Lending_Protocols/Aave/V3/Polygon/transactionsFinSurvival.csv", index=False)
print(transactions_df["timestamp"].max())

1755726959.0


In [11]:
import os
import pandas as pd

EVENTS = ["Deposit", "Withdraw", "Repay", "Borrow", "Liquidated"]
DATA_PATH = "./data/"

In [ ]:
users = {}
for index_event in EVENTS:
    for outcome_event in EVENTS:
        csv_path = os.path.join(DATA_PATH, index_event, outcome_event, "data.csv")
        if not os.path.exists(csv_path):
            continue
        df = pd.read_csv(csv_path)
        for _, row in df.iterrows():
            if row["user"] not in users:
                users[row["user"]] = {
                    "user_address": "0xUser123",
                    "description": "My custom user profile",
                    "transactions": [],
                }
            users[row["user"]]["transactions"].append(
                {"action": row[""], "symbol": "USDC", "amount": 5000, "timestamp": 0}
            )

In [4]:
for index_event in EVENTS:
    for outcome_event in EVENTS:
        if not (index_event == "Deposit" and outcome_event == "Withdraw"):
            continue
        csv_path = os.path.join(DATA_PATH, index_event, outcome_event, "data.csv")
        if not os.path.exists(csv_path):
            continue
        df = pd.read_csv(csv_path)
        for col in ["Index Event", "Outcome Event"]:
            if col in df.columns:
                mask = df[col].notna()
                df.loc[mask, col] = (
                    df.loc[mask, col]
                    .astype(str)
                    .str.replace("account liquidated", "liquidated", case=False, regex=False)
                    .str.strip()
                )
        df.to_csv(csv_path, index=False)
        print(f"Updated {csv_path}")

Updated ./data/Deposit/Withdraw/data.csv


In [5]:
for index_event in EVENTS:
    for outcome_event in EVENTS:
        if not (index_event == "Deposit" and outcome_event == "Withdraw"):
            continue
        csv_path = os.path.join(DATA_PATH, index_event, outcome_event, "data.csv")
        if not os.path.exists(csv_path):
            continue
        df = pd.read_csv(csv_path)
        for col in ["Index Event", "Outcome Event"]:
            if col in df.columns:
                mask = df[col].notna()
                df.loc[mask, col] = (
                    df.loc[mask, col]
                    .astype(str)
                    .str.replace("liquidation", "liquidated", case=False, regex=False)
                    .str.strip()
                )
        df.to_csv(csv_path, index=False)
        print(f"Updated {csv_path}")

Updated ./data/Deposit/Withdraw/data.csv


In [ ]:
for event in EVENTS:
    csv_path = os.path.join(DATA_PATH, event, event, "data.csv")
    if not (event == "Deposit" and event == "Withdraw"):
        continue
    if not os.path.exists(csv_path):
        continue
    df = pd.read_csv(csv_path)
    for col in ["Outcome Event"]:
        if col in df.columns:
            mask = df[col].notna()
            df.loc[mask, col] = (
                df.loc[mask, col]
                .astype(str)
                .str.replace(event + "_outcome", event, case=False, regex=False)
                .str.strip()
            )
    df.to_csv(csv_path, index=False)
    print(f"Updated {csv_path}")

In [9]:
for event in EVENTS:
    csv_path = os.path.join(DATA_PATH, event, event, "data.csv")
    if not (event == "Deposit" and event == "Withdraw"):
        continue
    if not os.path.exists(csv_path):
        continue
    df = pd.read_csv(csv_path)
    for col in ["Outcome Event"]:
        if col in df.columns:
            mask = df[col].notna()
            df.loc[mask, col] = (
                df.loc[mask, col]
                .astype(str)
                .str.replace(event, event.lower(), case=False, regex=False)
                .str.strip()
            )
    df.to_csv(csv_path, index=False)
    print(f"Updated {csv_path}")

In [12]:
index_event = "Liquidated"
for outcome_event in EVENTS:
    csv_path = os.path.join(DATA_PATH, index_event, outcome_event, "data.csv")
    if not os.path.exists(csv_path):
        continue
    df = pd.read_csv(csv_path)
    for col in ["Index Event", "Outcome Event"]:
        if col in df.columns:
            mask = df[col].notna()
            df.loc[mask, col] = (
                df.loc[mask, col]
                .astype(str)
                .str.replace("liquidation", "liquidated", case=False, regex=False)
                .str.strip()
            )
    df.to_csv(csv_path, index=False)
    print(f"Updated {csv_path}")

Updated ./data/Liquidated/Deposit/data.csv
Updated ./data/Liquidated/Withdraw/data.csv
Updated ./data/Liquidated/Repay/data.csv
Updated ./data/Liquidated/Borrow/data.csv
Updated ./data/Liquidated/Liquidated/data.csv


In [ ]:
import json
from collections import defaultdict

usersToTest = [
    "0xae0cc724fc85cd394435eb4f9ddfd3a8c088b4e2",
    "0x0e7b3aea0428137786edf20617670179435c530b",
    "0x97dc47d17ab56c32a4d2da1839e3ff261304e95d",
    "0x5091d57b4feb57db1da04b51cc5454f371b623a6",
]

for user in usersToTest:
    with open(
        f"../data/IDEA_DeFi_Research/Data/CSV/profile-generation/user_{user}.json"
    ) as f:
        userTest = json.load(f)

    userTransactions = userTest["transactions"]
    print(next(
            (e for i, e in enumerate(userTransactions) if (e["action"] == "Liquidated"))
        )['debt_symbol'])
    userTransactions = userTransactions[
        : next(
            (i for i, e in enumerate(userTransactions) if (e["action"] == "Liquidated")), -1
        )
    ]
    borrows = defaultdict(int)
    repays = defaultdict(int)
    for transaction in userTransactions:
        if transaction["action"] == "Borrow":
            borrows[transaction["symbol"]] += transaction["amount"]
        elif transaction["action"] == "Repay":
            repays[transaction["symbol"]] += transaction["amount"]
    print(f'{user}\'s "Debt" before liquidation: {[f'{symbol}: {borrows[symbol] - repays[symbol]}' for symbol in borrows.keys()]}')

WETH
0xae0cc724fc85cd394435eb4f9ddfd3a8c088b4e2's"Debt" before liquidation: ['USDC: 898.7291915312042', 'WETH: -1.6900000000003718e-06']
USDC
0x0e7b3aea0428137786edf20617670179435c530b's"Debt" before liquidation: ['USDC: 0.0', 'WETH: 75.7625662202381']
USDC
0x97dc47d17ab56c32a4d2da1839e3ff261304e95d's"Debt" before liquidation: ['USDC: 7.854323']
WETH
0x5091d57b4feb57db1da04b51cc5454f371b623a6's"Debt" before liquidation: ['USDC: 0.0']


In [ ]:
import pandas as pd

with open('../data/IDEA_DeFi_Research/Data/Lending_Protocols/Aave/V2/Polygon/users.csv') as f:
    old_users = pd.read_csv(f)['id']
old_users.head()

0    0x0000000000000000000000000000000000000001
1    0x0000000000000000000000000000000000001010
2    0x000000000000000000000000000000000000dead
3    0x0000000000085a12481aedb59eb3200332aca597
4    0x00000000000a29a0800f6f557ddbbe8249397de7
Name: id, dtype: object

In [10]:
with open('./data/users.csv') as f:
    users = pd.read_csv(f)['id']
new_users = set(users) - set(old_users)
print(f'Out of {len(users)} users, {len(new_users)} users were not among the {len(old_users)} old users.')

Out of 365677 users, 308205 users were not among the 1121518 old users.


In [ ]:
usersWithLiquidation = set()
outcome_event = "Liquidated"
for index_event in EVENTS:
    if index_event == outcome_event:
        continue
    csv_path = os.path.join(DATA_PATH, index_event, outcome_event, "data.csv")
    if not os.path.exists(csv_path):
        continue
    df = pd.read_csv(csv_path)
    usersWithLiquidation = usersWithLiquidation.union(set(df['user']))
    print(f"Got liquidated users from {csv_path}")
new_liquidated_users = set(usersWithLiquidation) - set(old_users)
print(f'Out of {len(usersWithLiquidation)} liquidated users, {len(new_liquidated_users)} liquidated users were not among the {len(old_users)} old users.')

Got liquidated users from ./data/Deposit/Liquidated/data.csv
Got liquidated users from ./data/Withdraw/Liquidated/data.csv
Got liquidated users from ./data/Repay/Liquidated/data.csv
Got liquidated users from ./data/Borrow/Liquidated/data.csv
Out of 259775 liquidated users, 204309 liquidated users were not among the 1121518 old users.


In [ ]:
usersWithLiquidation = set()
index_event = "Liquidated"
for outcome_event in EVENTS:
    if index_event == outcome_event:
        continue
    csv_path = os.path.join(DATA_PATH, index_event, outcome_event, "data.csv")
    if not os.path.exists(csv_path):
        continue
    df = pd.read_csv(csv_path)
    usersWithLiquidation = usersWithLiquidation.union(set(df['user']))
    print(f"Got liquidated users from {csv_path}")
new_liquidated_users = set(usersWithLiquidation) - set(old_users)
print(f'Out of {len(usersWithLiquidation)} liquidated users, {len(new_liquidated_users)} liquidated users were not among the {len(old_users)} old users.')

Got liquidated users from ./data/Liquidated/Deposit/data.csv
Got liquidated users from ./data/Liquidated/Withdraw/data.csv
Got liquidated users from ./data/Liquidated/Repay/data.csv
Got liquidated users from ./data/Liquidated/Borrow/data.csv
Out of 13743 liquidated users, 8899 liquidated users were not among the 1121518 old users.


In [16]:
[user for user in usersToTest if user in new_liquidated_users]

['0x97dc47d17ab56c32a4d2da1839e3ff261304e95d',
 '0x5091d57b4feb57db1da04b51cc5454f371b623a6']

In [17]:
import json

with open('./Aave-Simulator/results/zero-debt-liquidation-analysis/analysis_20251218_164135.json') as f:
    zero_debt_liquidated_users = [case['user_address'] for case in json.load(f)['zero_debt_cases']]
new_zero_liquidated_users = set(zero_debt_liquidated_users) - set(old_users)
print(f'Out of {len(zero_debt_liquidated_users)} zero-debt liquidated users, {len(new_zero_liquidated_users)} zero-debt liquidated users were not among the {len(old_users)} old users.')

Out of 1328 zero-debt liquidated users, 366 zero-debt liquidated users were not among the 1121518 old users.


In [1]:
from actionAgentTraining import verify_amount_feature_effect, get_model_for_pair_and_date
import pandas as pd

model = get_model_for_pair_and_date("Repay", "Liquidated", pd.to_datetime("2024-05-05 15:33:51.800000"))
print(model)

KeyboardInterrupt: 

In [3]:
from utils.constants import MODEL_CACHE_DIR
import os
import pickle
import numbers
import numpy as np

def scan_baseline_clips(print_fn=print, clip_thresholds=None):
    """Scan all *_baseline.pkl files in `MODEL_CACHE_DIR` and report values equal
    to clipping thresholds (by default -50 and 50).

    Returns a dict mapping filename -> list of (path, value) where value was equal
    to a clipping threshold.
    """
    if clip_thresholds is None:
        clip_thresholds = [-50.0, 50.0]

    results = {}
    for fname in os.listdir(MODEL_CACHE_DIR):
        if not fname.endswith("_baseline.pkl"):
            continue
        full = os.path.join(MODEL_CACHE_DIR, fname)
        try:
            with open(full, "rb") as f:
                obj = pickle.load(f)
        except Exception as e:
            print_fn(f"Failed to load {full}: {e}")
            continue

        found = []

        def _walk(o, path="root"):
            # Numeric scalar
            if isinstance(o, numbers.Number):
                for t in clip_thresholds:
                    try:
                        if float(o) == float(t):
                            found.append((path, float(o)))
                    except Exception:
                        pass
            # numpy arrays
            elif isinstance(o, np.ndarray):
                for t in clip_thresholds:
                    mask = np.isclose(o, t)
                    if np.any(mask):
                        idxs = np.argwhere(mask)
                        for idx in idxs:
                            found.append((f"{path}{tuple(idx.tolist())}", float(t)))
            # list/tuple/set
            elif isinstance(o, (list, tuple, set)):
                for i, v in enumerate(o):
                    _walk(v, f"{path}[{i}]")
            # dict
            elif isinstance(o, dict):
                for k, v in o.items():
                    _walk(v, f"{path}.{k}")
            else:
                # try to inspect attributes (e.g., objects with __dict__)
                if hasattr(o, "__dict__"):
                    for k, v in vars(o).items():
                        _walk(v, f"{path}.{k}")

        _walk(obj)

        if found:
            results[full] = found
            print_fn(f"Found clipped values in {full}:")
            for p, v in found:
                print_fn(f"  {p}: {v}")

    if not results:
        print_fn("No clipped values found in any _baseline.pkl files.")

    return results

scan_baseline_clips()

No clipped values found in any _baseline.pkl files.


{}

In [1]:
from actionAgentTraining import get_user_history
import pandas as pd
import os
from tqdm import tqdm

def compute_history_lengths(train_csv_path=os.path.join('cache', 'train_set.csv'), save_to=None, max_rows=None, show_progress=True):
    """Compute, for every transaction in `train_set.csv`, how long the user's
    transaction history is up to that transaction timestamp. Returns a DataFrame with:
      - user, timestamp, history_count, history_span_seconds
    Parameters:
      - train_csv_path: path to CSV of training transactions (default 'cache/train_set.csv')
      - save_to: optional path to save the output CSV
      - max_rows: optional int to limit processing for quick checks
      - show_progress: print progress while iterating
    """
    df = pd.read_csv(train_csv_path)
    if max_rows is not None:
        df = df.head(max_rows)

    rows = []
    total = len(df)
    for i, row in tqdm(df.iterrows(), total=total, disable=not show_progress):
        user = row.get('user') if 'user' in row.index else row.get('User')
        ts = int(row.get('timestamp') if 'timestamp' in row.index else row.get('Timestamp'))
        try:
            hist = get_user_history(user, ts)
        except Exception as e:
            hist = None
        count = 0
        span = 0
        if isinstance(hist, pd.DataFrame) and not hist.empty:
            count = len(hist)
            if 'timestamp' in hist.columns:
                try:
                    earliest = int(hist['timestamp'].min())
                    span = ts - earliest
                except Exception:
                    span = None
            else:
                span = None
        rows.append({'index': i, 'user': user, 'timestamp': ts, 'history_count': count, 'history_span_seconds': span})
        # if show_progress and (i % 1000 == 0):
        #     print(f'Processed {i+1}/{total} transactions')

    res = pd.DataFrame(rows)
    if save_to is not None:
        res.to_csv(save_to, index=False)
    return res

# Example: run over full train set and save summary to cache/history_lengths.csv
# Uncomment to execute:
out = compute_history_lengths()
out.head()


100%|██████████| 2000/2000 [1:04:13<00:00,  1.93s/it]


,index,user,timestamp,history_count,history_span_seconds
0,0,0xcfdaf25cdfd9c68f37ced1408067d032fc9ec68b,1727636330,883,79050596
1,1,0x0fb6eb88f429e2912c3117feeaf512fe35d6ad04,1715218763,503204,1552188
2,2,0x357aa453a873ef716ea69088c85be4adbda6282e,1719850422,150380,19213983
3,3,0x18fbd380ad51a4bcb3a1a8a89107483d10b716c0,1714940836,341801,14907190
4,4,0x706add824132b2a17142ed3c613d42e9c2db635f,1703365296,75,4400069


In [9]:
out.sort_values(by=['history_count', 'history_span_seconds'], ascending=True).head(50)

,index,user,timestamp,history_count,history_span_seconds
13,13,0x90eaa542a9575358b24bd7f5b8721989298c546a,1694019124,5,0
41,41,0x491919bcf7704fb4c328dd84affd2ad8eca5fd6a,1697124486,5,0
65,65,0x3f5282adca3b9bc02d385e539ad0e66ee7f65fce,1691929436,5,0
67,67,0x309a33959bd2cda19473110db175e6a8595cae4b,1701253162,5,0
77,77,0x111111cab58e134a2e3ecd8d3fb72488cf01fefa,1691318969,5,0
106,106,0xe964182ebe3aeb55fed219a07a6e3f38a59c1339,1692185020,5,0
110,110,0x54faad784c3addef84adecaa5ee30d3ed9abcf20,1717535235,5,0
112,112,0xa2a833e55401aab11ddf0006c369d3cea1d11f55,1717843333,5,0
114,114,0x25d963e67f5570cea9de9651111fd8466d27e598,1725511909,5,0
115,115,0xde3621b8ecd32273eb12f8b5bd1d5d7ea04b91b6,1703720689,5,0


In [8]:
len(out[out['history_count'] < 10000])

1039

In [7]:
out['history_count'].median()

np.float64(6985.0)

In [1]:
import subprocess


def getTotalTimestamps(filename, max_count=None, reverse=False):
    match_string = '^Total timestamps:'
    if reverse:
        process = ["tac", filename, "|", "grep", match_string]
    else:
        process = ["grep", match_string, filename]
    if max_count is not None:
        process += ["-m", str(max_count)]
    return [
        int(match[len(match_string) - 1 :])
        for match in subprocess.check_output(process, text=True).split(
            "\n"
        )[:-1]
    ]


badOutputCounts = getTotalTimestamps("outputSpeedTest4.out", max_count=10)
goodOutputCounts = getTotalTimestamps(
    "outputSpeedTest6.out", max_count=10)

CalledProcessError: Command '['grep', '^Total timestamps:', 'outputSpeedTest6.out', '-m', '10']' returned non-zero exit status 1.

In [ ]:
if len(goodOutputCounts) < len(badOutputCounts):
    badOutputCounts = badOutputCounts[:len(goodOutputCounts)]
badSum = sum(badOutputCounts)
goodSum = sum(goodOutputCounts)
print(f"badOutputCounts sum: {badSum}\ngoodOutputCounts sum: {goodSum}")
print(f"ratio: {goodSum/badSum}")

badOutputCounts sum: 4468
goodOutputCounts sum: 1202605
ratio: 269.15957923008057


In [ ]:
import json

with open("./Aave-Simulator/data/reserves/price_history.json", "r") as file:
    price_history = json.load(file)

In [6]:
print(f"price_history length: {sum([len(prices) for prices in price_history.values()])}")

price_history length: 7437460


In [ ]:
print(f"price_history keys: {price_history.keys()}")

price_history keys: dict_keys(['DAI', 'USDC', 'WPOL', 'jEUR', 'WETH', 'LINK', 'GHST', 'USDT', 'WBTC', 'agEUR', 'CRV', 'BAL', 'EURS', 'DPI', 'SUSHI', 'miMATIC', 'MaticX', 'wstETH', 'AAVE', 'stMATIC'])


In [13]:
allTimestamps = [timestamp for timestamps_pairs in price_history.values() for timestamp in timestamps_pairs.keys()]
len(allTimestamps)

7437460

In [12]:
uniqueTimestamps = set(allTimestamps)
len(uniqueTimestamps)

6011909

In [12]:
import subprocess
import re

def getSecondsTaken(filename):
    with open(filename, 'r') as f:
        text = f.read()
    pattern = r"^Name:([^\n]+)\nSeconds Taken:([^\n]+)"
    matches = re.findall(pattern, text, re.MULTILINE)
    names = set(name for name, _ in matches)
    seconds_dict = {}
    for name in names:
        seconds_dict[name] = [float(seconds) for subName, seconds in matches if subName == name]
    return seconds_dict

In [13]:
seconds_dict = getSecondsTaken("./speedTest31Output.out")

In [14]:
for name in seconds_dict.keys():
    print(f"Max seconds taken for {name}: {max(seconds_dict[name])}")
    print(f"Average seconds taken for {name}: {sum(seconds_dict[name])/len(seconds_dict[name])}")
print(f"Average seconds per group: {sum(sum(seconds_dict[name])/len(seconds_dict[name]) for name in seconds_dict)}")

Max seconds taken for adaptive_granularity: 8.13046669960022
Average seconds taken for adaptive_granularity: 0.6540547313818843
Max seconds taken for event_driven: 104.71600556373596
Average seconds taken for event_driven: 12.133959152244195
Max seconds taken for binary_search: 6.102752447128296
Average seconds taken for binary_search: 0.33750013115032124
Max seconds taken for hybrid: 91.93166089057922
Average seconds taken for hybrid: 10.228341040281583
Max seconds taken for model_based: 0.752084493637085
Average seconds taken for model_based: 0.060829215266853316
Average seconds per group: 23.414684270324837


In [9]:
import subprocess
import re
from datetime import datetime

def getSecondsTaken(filename):
    with open(filename, 'r') as f:
        text = f.read()
    pattern = r"(\d\d\d\d-\d\d\-\d\d \d\d:\d\d:\d\d,\d\d\d) DEBUG Checkpoint 12$"
    matches = re.findall(pattern, text, re.MULTILINE)
    timestamps = [datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S,%f").timestamp() for date_string in matches]
    return timestamps

In [19]:
timestamps = getSecondsTaken("./speedTest32.log")
# timestamps

In [20]:
print(f"Total seconds to run: {timestamps[-1]-timestamps[0]}")
print(f"Total hours to run: {(timestamps[-1]-timestamps[0])/60/60}")
print(f"Average seconds per test: {(timestamps[-1]-timestamps[0])/(len(timestamps)-1)}")

Total seconds to run: 35672.6210000515
Total hours to run: 9.909061388903194
Average seconds per test: 4.127342473684079
